<a href="https://colab.research.google.com/github/stefanvanberkum/CD-ABSC/blob/main/getBERT_laptop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
For use on local runtime.

How to run:
- Download the appropriate BERT model from: 
  https://github.com/google-research/bert (here BERT-Base).

- Create a virtual environment (e.g. using anaconda).

- Install tensorflow in your virtual environment (pip install tensorflow==1.15).

- Start a local runtime in your virtual environment using
  https://research.google.com/colaboratory/local-runtimes.html

- Make sure the BERT model is in your system path (here named
  'uncased_L-12_H-768_A-12').

- Make sure all data is available and update the paths at the end of this code.

Adapted from Trusca, Wassenberg, Frasincar and Dekker (2020) for use on a local
runtime

Truşcǎ M.M., Wassenberg D., Frasincar F., Dekker R. (2020) A Hybrid Approach for 
Aspect-Based Sentiment Analysis Using Deep Contextual Word Embeddings and 
Hierarchical Attention. In: Bielikova M., Mikkonen T., Pautasso C. (eds) Web 
Engineering. ICWE 2020. Lecture Notes in Computer Science, vol 12128. Springer, 
Cham. https://doi.org/10.1007/978-3-030-50578-3_25

https://github.com/mtrusca/HAABSA_PLUS_PLUS
'''

"\nFor use on local runtime.\n\nHow to run:\n- Download the appropriate BERT model from: \n  https://github.com/google-research/bert (here BERT-Base).\n\n- Create a virtual environment (e.g. using anaconda).\n\n- Install tensorflow in your virtual environment (pip install tensorflow==1.15).\n\n- Start a local runtime in your virtual environment using\n  https://research.google.com/colaboratory/local-runtimes.html\n\n- Make sure the BERT model is in your system path (here named\n  'uncased_L-12_H-768_A-12').\n\n- Make sure all data is available and update the paths at the end of this code.\n\nAdapted from Trusca, Wassenberg, Frasincar and Dekker (2020) for use on a local\nruntime\n\nTruşcǎ M.M., Wassenberg D., Frasincar F., Dekker R. (2020) A Hybrid Approach for \nAspect-Based Sentiment Analysis Using Deep Contextual Word Embeddings and \nHierarchical Attention. In: Bielikova M., Mikkonen T., Pautasso C. (eds) Web \nEngineering. ICWE 2020. Lecture Notes in Computer Science, vol 12128. S

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys

sys.path.append('bert-master/')

import codecs
import collections
import json
import re
import os
import pprint
import numpy as np
import tensorflow as tf

import modeling
import tokenization

In [ ]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = 'uncased_L-12_H-768_A-12'
LAYERS = [-1, -2, -3, -4]
NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 87
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128

In [ ]:
class InputExample(object):

    def __init__(self, unique_id, text_a, text_b=None):
        self.unique_id = unique_id
        self.text_a = text_a
        self.text_b = text_b


In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
        self.unique_id = unique_id
        self.tokens = tokens
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.input_type_ids = input_type_ids


In [ ]:
def input_fn_builder(features, seq_length):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    all_unique_ids = []
    all_input_ids = []
    all_input_mask = []
    all_input_type_ids = []

    for feature in features:
        all_unique_ids.append(feature.unique_id)
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_input_type_ids.append(feature.input_type_ids)

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        num_examples = len(features)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "unique_ids":
                tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_type_ids":
                tf.constant(
                    all_input_type_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
        })

        d = d.batch(batch_size=batch_size, drop_remainder=False)
        return d

    return input_fn


In [ ]:
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        unique_ids = features["unique_ids"]
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        input_type_ids = features["input_type_ids"]

        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=input_ids,
            input_mask=input_mask,
            token_type_ids=input_type_ids,
            use_one_hot_embeddings=use_one_hot_embeddings)

        if mode != tf.estimator.ModeKeys.PREDICT:
            raise ValueError("Only PREDICT modes are supported: %s" % (mode))

        tvars = tf.trainable_variables()
        scaffold_fn = None
        (assignment_map,
         initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
            tvars, init_checkpoint)
        if use_tpu:

            def tpu_scaffold():
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                return tf.train.Scaffold()

            scaffold_fn = tpu_scaffold
        else:
            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)

        all_layers = model.get_all_encoder_layers()

        predictions = {
            "unique_id": unique_ids,
        }

        for (i, layer_index) in enumerate(layer_indexes):
            predictions["layer_output_%d" % i] = all_layers[layer_index]

        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
        return output_spec

    return model_fn


In [ ]:
def convert_examples_to_features(examples, seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > seq_length - 2:
                tokens_a = tokens_a[0:(seq_length - 2)]

        tokens = []
        input_type_ids = []
        tokens.append("[CLS]")
        input_type_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            input_type_ids.append(0)
        tokens.append("[SEP]")
        input_type_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                input_type_ids.append(1)
            tokens.append("[SEP]")
            input_type_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < seq_length:
            input_ids.append(0)
            input_mask.append(0)
            input_type_ids.append(0)

        assert len(input_ids) == seq_length
        assert len(input_mask) == seq_length
        assert len(input_type_ids) == seq_length

        if ex_index < 5:
            tf.logging.info("*** Example ***")
            tf.logging.info("unique_id: %s" % (example.unique_id))
            tf.logging.info("tokens: %s" % " ".join(
                [tokenization.printable_text(x) for x in tokens]))
            tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            tf.logging.info(
                "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

        features.append(
            InputFeatures(
                unique_id=example.unique_id,
                tokens=tokens,
                input_ids=input_ids,
                input_mask=input_mask,
                input_type_ids=input_type_ids))
    return features


In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


In [ ]:
def read_sequence(input_sentences):
    examples = []
    unique_id = 0
    for sentence in input_sentences:
        line = tokenization.convert_to_unicode(sentence)
        examples.append(InputExample(unique_id=unique_id, text_a=line))
        unique_id += 1
    return examples


In [ ]:
def get_features(input_text, dim=768):
    tf.logging.set_verbosity(tf.logging.ERROR)

    layer_indexes = LAYERS

    bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG);

    tokenizer = tokenization.FullTokenizer(
        vocab_file=VOCAB_FILE, do_lower_case=True)

    examples = read_sequence(input_text)

    features = convert_examples_to_features(
        examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)

    unique_id_to_feature = {}
    for feature in features:
        unique_id_to_feature[feature.unique_id] = feature

    model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint=INIT_CHECKPOINT,
        layer_indexes=layer_indexes,
        use_tpu=False,
        use_one_hot_embeddings=True)

    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
        model_dir=BERT_PRETRAINED_DIR,
        use_tpu=False,
        model_fn=model_fn,
        config=tf.contrib.tpu.RunConfig(),
        predict_batch_size=BATCH_SIZE,
        train_batch_size=BATCH_SIZE)

    input_fn = input_fn_builder(
        features=features, seq_length=MAX_SEQ_LENGTH)

    # Get features
    for result in estimator.predict(input_fn, yield_single_examples=True):
        unique_id = int(result["unique_id"])
        feature = unique_id_to_feature[unique_id]
        output = collections.OrderedDict()
        for (i, token) in enumerate(feature.tokens):
            layers = []
            for (j, layer_index) in enumerate(layer_indexes):
                layer_output = result["layer_output_%d" % j]
                layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
                layers.append(layer_output_flat)
            output[token] = sum(layers)[:dim]

    return output


In [ ]:
# When it takes too long, data can be split in multiple subfiles such as in
# lines 5-30
lines = open('data/programGeneratedData/BERT/laptop/raw_data_laptop_train_2014.txt', errors='replace').readlines()


for j in range(0, len(lines), 150):
  with open("BERT_base_laptop_2014_" + str(round(j/150)) + ".txt", 'w') as f:
      for i in range(j, j + 150, 3):  # Was 0*3, 2530*3, 3
          print("sentence: " + str(i / 3) + " out of " + str(len(lines) / 3) + " in " + "raw_data;")
          target = lines[i + 1].lower().split()
          words = lines[i].lower().split()
          words_l, words_r = [], []
          flag = True
          for word in words:
              if word == '$t$':
                  flag = False
                  continue
              if flag:
                  words_l.append(word)
              else:
                  words_r.append(word)
          sentence = " ".join(words_l + target + words_r)
          print(sentence)
          embeddings = get_features([sentence])

          for key, value in embeddings.items():
              f.write('\n%s ' % key)
              for v in value:
                f.write('%s ' % v)

'''
with open("data/externalData/BERT_base_laptop_2014.txt", 'w') as f:
  for i in range(0, len(lines), 3):  # Was 0*3, 2530*3, 3
    print("sentence: " + str(i / 3) + " out of " + str(len(lines) / 3) + " in " + "raw_data;")
    target = lines[i + 1].lower().split()
    words = lines[i].lower().split()
    words_l, words_r = [], []
    flag = True
    for word in words:
      if word == '$t$':
        flag = False
        continue
      if flag:
        words_l.append(word)
      else:
        words_r.append(word)
    sentence = " ".join(words_l + target + words_r)
    print(sentence)
    embeddings = get_features([sentence])

    for key, value in embeddings.items():
      f.write('\n%s ' % key)
      for v in value:
        f.write('%s ' % v)
'''

sentence: 0.0 out of 2313.0 in raw_data;
i charge it at night and skip taking the cord with me because of the good battery life .
sentence: 1.0 out of 2313.0 in raw_data;
i charge it at night and skip taking the cord with me because of the good battery life .
sentence: 2.0 out of 2313.0 in raw_data;
the tech guy then said the service center does not do 1-to-1 exchange and i have to direct my concern to the `` sales '' team , which is the retail shop which i bought my netbook from .
sentence: 3.0 out of 2313.0 in raw_data;
the tech guy then said the service center does not do 1-to-1 exchange and i have to direct my concern to the `` sales '' team , which is the retail shop which i bought my netbook from .
sentence: 4.0 out of 2313.0 in raw_data;
the tech guy then said the service center does not do 1-to-1 exchange and i have to direct my concern to the `` sales '' team , which is the retail shop which i bought my netbook from .
sentence: 5.0 out of 2313.0 in raw_data;
it is of high qual

IndexError: ignored